## Problem Statement : Hospital Patient Data Analysis
Context:
A hospital maintains patient records including admission details, department, diagnosis, doctor, and bill amount. You have two datasets: one with patient info and another with billing details. Some patients have blank bill amounts, and there are multiple rows for the same patient due to follow-ups.


1.	Load the patient dataset and show summary with info().

In [4]:
import pandas as pd
df1=pd.read_csv("Billing_Data.csv")
#df2=pd.read_csv("Patient_Data.csv")
df1.head()


,PatientID,InsuranceCovered,FinalAmount
0,101,2000,3000
1,102,1500,3500
2,103,2500,5000
3,104,3000,3200
4,105,1000,4000


In [5]:
df2=pd.read_csv("Patient_Data.csv")

df2.head()

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,NaN,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,NaN,2,2023-01-14 08:45


In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   PatientID         5 non-null      int64
 1   InsuranceCovered  5 non-null      int64
 2   FinalAmount       5 non-null      int64
dtypes: int64(3)
memory usage: 252.0 bytes


In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


2.	Select only the columns relevant for billing: ['PatientID', 'Department', 'Doctor', 'BillAmount'].

In [8]:
billing_columns=df2[["PatientID","Department","Doctor","BillAmount"]]
print(billing_columns)

   PatientID   Department     Doctor  BillAmount
0        101   Cardiology  Dr. Smith      5000.0
1        102    Neurology   Dr. John         NaN
2        103  Orthopedics    Dr. Lee      7500.0
3        104   Cardiology  Dr. Smith      6200.0
4        105  Dermatology   Dr. Rose         NaN
5        101   Cardiology  Dr. Smith      5000.0


3.	Drop administrative columns like ['ReceptionistID', 'CheckInTime'].

In [9]:
df_clean=df2.drop(columns=["ReceptionistID","CheckInTime"])
df_clean

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN
5,101,Alice,Cardiology,Dr. Smith,5000.0


4.	Use groupby to find total bill amount per department.

In [14]:
department_total=df_clean.groupby('Department')['BillAmount'].sum()
print(department_total)

Department
Cardiology     16200.0
Dermatology        0.0
Neurology          0.0
Orthopedics     7500.0
Name: BillAmount, dtype: float64


5.	Remove duplicate patient records based on PatientID.

In [15]:
df_unique=df_clean.drop_duplicates(subset='PatientID')
df_unique

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN


6.	Fill missing BillAmount values with the mean bill amount.

In [18]:
mean_bill=df_unique['BillAmount'].mean()
df_unique['BillAmount']=df_clean['BillAmount'].fillna(mean_bill)
df_unique

C:\Users\vanam\AppData\Local\Temp\ipykernel_21576\2489212546.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['BillAmount']=df_clean['BillAmount'].fillna(mean_bill)


,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,5925.0
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,5925.0


7.	Merge the billing dataset with patient dataset on PatientID.

In [19]:
merge_df=df_unique.merge(df1,on='PatientID',how='left')
merge_df

,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0,2000,3000
1,102,Bob,Neurology,Dr. John,5925.0,1500,3500
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,2500,5000
3,104,David,Cardiology,Dr. Smith,6200.0,3000,3200
4,105,Eva,Dermatology,Dr. Rose,5925.0,1000,4000


8.	Concatenate an additional DataFrame that contains new patients for the current week (row-wise).

In [20]:
new_records=pd.DataFrame({
    'PatientID':[106,107],
    'Name':['John','Kim'],
    'Department':['Cardiology','Neurology'],
    'Doctor':['Dr.Smith','Dr.John'],
    'BillAmount':[5500,6000]
})
df_extend=pd.concat([merge_df,new_records],axis=0)
df_extend

,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0,2000.0,3000.0
1,102,Bob,Neurology,Dr. John,5925.0,1500.0,3500.0
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,2500.0,5000.0
3,104,David,Cardiology,Dr. Smith,6200.0,3000.0,3200.0
4,105,Eva,Dermatology,Dr. Rose,5925.0,1000.0,4000.0
0,106,John,Cardiology,Dr.Smith,5500.0,NaN,NaN
1,107,Kim,Neurology,Dr.John,6000.0,NaN,NaN


9.	Concatenate new billing category columns like ['InsuranceCovered', 'FinalAmount'] (column-wise).

In [21]:
new_cols = pd.DataFrame({
    'InsuranceCovered': [1800,2500,1900,1860,2400,2610,1890],
    'FinalAmount': [4700,12000,2600,3900,8000,3800,6100]
})

df_final = pd.concat([df_extend.reset_index(drop=True), new_cols], axis=1)
print(df_final)

   PatientID     Name   Department     Doctor  BillAmount  InsuranceCovered  \
0        101    Alice   Cardiology  Dr. Smith      5000.0            2000.0   
1        102      Bob    Neurology   Dr. John      5925.0            1500.0   
2        103  Charlie  Orthopedics    Dr. Lee      7500.0            2500.0   
3        104    David   Cardiology  Dr. Smith      6200.0            3000.0   
4        105      Eva  Dermatology   Dr. Rose      5925.0            1000.0   
5        106     John   Cardiology   Dr.Smith      5500.0               NaN   
6        107      Kim    Neurology    Dr.John      6000.0               NaN   

   FinalAmount  InsuranceCovered  FinalAmount  
0       3000.0              1800         4700  
1       3500.0              2500        12000  
2       5000.0              1900         2600  
3       3200.0              1860         3900  
4       4000.0              2400         8000  
5          NaN              2610         3800  
6          NaN              189